In [ ]:
import torch
import numpy as np

In [ ]:
from seasalt.salt_net import (
    train_denoiser,
    train_noise_detector,
    Desnoiser,
    NoiseDetector,
    NoiseType,
    get_test_dataloader,
    get_train_dataloader,
)

In [ ]:
torch.manual_seed(101)
np.random.seed(101)

if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

device

In [ ]:
train_dataloader = get_train_dataloader(
    NoiseType.SAP, min_noise=0.5, max_noise=0.5, batch_size=8
)
val_dataloader = get_test_dataloader(
    NoiseType.SAP, min_noise=0.5, max_noise=0.5, batch_size=8
)

In [ ]:
model = NoiseDetector()
train_noise_detector(
    model,
    1e-3,
    train_dataloader,
    val_dataloader,
    device,
    "noise_detector_refactor_test",
    100,
    True,
)

In [ ]:
model = Desnoiser()
model = model.to(device)
train_denoiser(
    model,
    1e-3,
    train_dataloader,
    val_dataloader,
    device,
    "denoiser_refactor_test",
    100,
    True,
)